## Initial setup

In [ ]:
import pandas as pd
import numpy as np

import google.cloud.aiplatform as aiplatform
from google.cloud.aiplatform import model_monitoring

In [ ]:
# Replace values acc. to your project/bucket
PROJECT_ID = "<PROJECT_ID>"
REGION = "us-central1"

In [ ]:
aiplatform.init(project=PROJECT_ID,
                location=REGION)

Copy-paste endpoint info from the AutoML notebook e.g. look for something like "projects/123/locations/us-central1/endpoints/456"

In [ ]:
endpoint = aiplatform.Endpoint('<endpoint-info>')

## Configure alerting specification

In [ ]:
USER_EMAIL = "<email-of-alert-recipient>"

alerting_config = model_monitoring.EmailAlertConfig(user_emails=[USER_EMAIL],
                                                    enable_logging=True)

In [ ]:
MONITOR_INTERVAL = 1  # least count = 1; measured in hours

# Create schedule configuration
schedule_config = model_monitoring.ScheduleConfig(monitor_interval=MONITOR_INTERVAL)

In [ ]:
SAMPLE_RATE = 0.95  # default value is 0.8 i.e. 80%

# Create sampling configuration
logging_sampling_strategy = model_monitoring.RandomSampleConfig(sample_rate=SAMPLE_RATE)

In [ ]:
DRIFT_THRESHOLD_VALUE = 0.05  # default value is 0.3

# Set column-wise threshold values
DRIFT_THRESHOLDS = {"<column1-name>": DRIFT_THRESHOLD_VALUE,
                    "<column2-name>": DRIFT_THRESHOLD_VALUE,
                    "<column3-name>": DRIFT_THRESHOLD_VALUE,
                    "<column4-name>": DRIFT_THRESHOLD_VALUE,
                    "<column5-name>": DRIFT_THRESHOLD_VALUE}

drift_config = model_monitoring.DriftDetectionConfig(drift_thresholds=DRIFT_THRESHOLDS)

In [ ]:
TRAIN_DATA_GCS_URI = "<GCS-path>"  # source of training csv file
TARGET = "<column-name>"  # label column

SKEW_THRESHOLD_VALUE = 0.05  # default value is 0.3

SKEW_THRESHOLDS = {"<column1-name>": SKEW_THRESHOLD_VALUE,
                   "<column2-name>": SKEW_THRESHOLD_VALUE,
                   "<column3-name>": SKEW_THRESHOLD_VALUE,
                   "<column4-name>": SKEW_THRESHOLD_VALUE,
                   "<column5-name>": SKEW_THRESHOLD_VALUE}

skew_config = model_monitoring.SkewDetectionConfig(data_source=TRAIN_DATA_GCS_URI,
                                                   skew_thresholds=SKEW_THRESHOLDS,
                                                   target_field=TARGET,
                                                   data_format="csv")

In [ ]:
objective_config = model_monitoring.ObjectiveConfig(
                                                    skew_detection_config=skew_config,
                                                    drift_detection_config=drift_config,
                                                   )

## Monitoring

In [ ]:
monitoring_job = aiplatform.ModelDeploymentMonitoringJob.create(
                                                                display_name="<your-display-name>",  # for GCP console
                                                                project=PROJECT_ID,
                                                                location=REGION,
                                                                endpoint=endpoint,
                                                                logging_sampling_strategy=logging_sampling_strategy,
                                                                schedule_config=schedule_config,
                                                                alert_config=alerting_config,
                                                                objective_configs=objective_config,
                                                               )

print(monitoring_job)

Check current status

In [ ]:
jobs = monitoring_job.list(filter="display_name=<your-display-name>")  # same as in previous cell
job = jobs[0]
print(job.state)  # might show pending at the beginning

Wait for a few minutes and check again

In [ ]:
print(job.state)  # wait till it shows running

Generate synthetic data for prediction requests

In [ ]:
import random

instances = []

# Set appropriate values, based on each column
for _ in range(1000):
    new_row = {
                "age": "",
                "workclass": "",
                "fnlwgt": "",
                "education": "",
                "education_num": "",
                "marital_status": "",
                "occupation": "",
                "relationship": "",
                "race": "",
                "sex": "",
                "capital_gain": "",
                "capital_loss": "",
                "hours_per_week": "",
                "native_country": ""
              }
    instances.append(new_row)

In [ ]:
for instance in instances:
    response = endpoint.predict(instances=[instance])

prediction = response[0]

# print the prediction for the first instance
print(prediction)

In [ ]:
instances = []

# Set appropriate values, based on each column
for _ in range(10):
    new_row = {
                "age": ",
                "workclass": "",
                "fnlwgt": "",
                "education": "",
                "education_num": "",
                "marital_status": "",
                "occupation": "",
                "relationship": "",
                "race": "",
                "sex": "",
                "capital_gain": "",
                "capital_loss": "",
                "hours_per_week": "",
                "native_country": ""
              }
    instances.append(new_row)

In [ ]:
for instance in instances:
    response = endpoint.predict(instances=[instance])

prediction = response[0]

# print the prediction for the first instance
print(prediction)

Check Vertex AI Endoints (Monitoring) console, to check latest monitoring run - it runs only at regular intervals e.g. every 1 hour, not all the time - of the deployed model and check for alerts. After some time, alerts should show up like this..

<img src="GCP_monitoring_console_alert.jpg" width=1000 height=600 />

And a corresponding email as well e.g. ..

<img src="GCP_monitoring_email_alert.jpg" width=1000 height=600 />